In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Paraíba - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_PB.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Consumo de Cimento (t)
0,2003-1,8.315958,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,118.926160,0.182304,0.656510,3.017292e+07,1.631334e+06,7.291169,2.736342e+07,38.426
1,2003-2,8.309895,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,117.724891,0.185909,0.656949,3.021010e+07,1.632950e+06,7.293438,2.737516e+07,31.672
2,2003-3,8.303831,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,118.501965,0.189593,0.657388,3.024728e+07,1.634566e+06,7.295708,2.738689e+07,28.775
3,2003-4,8.297767,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,119.390679,0.193353,0.657827,3.028445e+07,1.636182e+06,7.297978,2.739862e+07,28.571
4,2003-5,8.291704,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,121.517541,0.197186,0.658266,3.032163e+07,1.637798e+06,7.300247,2.741035e+07,31.613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294.165653,0.679715,NaN,NaN,NaN,NaN,NaN,95.860
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.457511,0.678961,NaN,NaN,NaN,NaN,NaN,90.790
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292.809622,0.677475,NaN,NaN,NaN,NaN,NaN,94.588
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290.980478,0.676277,NaN,NaN,NaN,NaN,NaN,89.777


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
0,-0.800943,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.092830,-2.424081,-2.168844,-1.710694,-2.484291,-2.270808,-2.124240
1,-0.805366,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.140285,-2.392248,-2.133132,-1.692339,-2.436666,-2.234559,-2.091568
2,-0.809789,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.109587,-2.359718,-2.097421,-1.673984,-2.389041,-2.198310,-2.058896
3,-0.814212,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.074479,-2.326517,-2.061709,-1.655629,-2.341417,-2.162061,-2.026224
4,-0.818635,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.990458,-2.292670,-2.025998,-1.637274,-2.293792,-2.125812,-1.993552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.272641,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.885384,0.924040,1.235988,1.087341,-0.170948,0.675345,0.728937
188,1.277530,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.959697,0.934231,1.228601,1.077496,-0.163231,0.658514,0.715410
189,1.282419,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,2.094602,0.957607,1.221214,1.067651,-0.155514,0.641683,0.701882
190,1.287308,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,2.203705,0.987746,1.213827,1.057806,-0.147797,0.624852,0.688355


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      31.891
1      26.089
2      32.066
3      26.351
4      27.348
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Paraíba - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
0,-0.800943,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.092830,-2.424081,-2.168844,-1.710694,-2.484291,-2.270808,-2.124240
1,-0.805366,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.140285,-2.392248,-2.133132,-1.692339,-2.436666,-2.234559,-2.091568
2,-0.809789,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.109587,-2.359718,-2.097421,-1.673984,-2.389041,-2.198310,-2.058896
3,-0.814212,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.074479,-2.326517,-2.061709,-1.655629,-2.341417,-2.162061,-2.026224
4,-0.818635,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.990458,-2.292670,-2.025998,-1.637274,-2.293792,-2.125812,-1.993552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.300589,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.747301,0.928050,1.431308,1.193935,-0.099354,0.924381,1.014355
158,1.292964,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.751028,0.933525,1.429336,1.195558,-0.105914,0.923232,1.010769
159,1.285339,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.748338,0.939401,1.427365,1.197181,-0.112475,0.922084,1.007184
160,1.277715,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.708461,0.946049,1.425393,1.198804,-0.119036,0.920935,1.003598


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      31.891
1      26.089
2      32.066
3      26.351
4      27.348
        ...  
157    58.962
158    73.459
159    59.150
160    64.833
161    63.840
Name: Paraíba - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
126,0.913576,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.690698,0.725920,0.453223,0.844953,0.848711,1.030555,1.033347
127,0.936878,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.689517,0.735237,0.463389,0.864872,0.830125,1.045296,1.048217
128,0.960180,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.699239,0.744336,0.473555,0.884791,0.811540,1.060038,1.063087
129,0.983482,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.679647,0.753547,0.483721,0.904710,0.792954,1.074779,1.077957
130,1.006783,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.659340,0.762358,0.493887,0.924630,0.774368,1.089520,1.092827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.272641,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.885384,0.924040,1.235988,1.087341,-0.170948,0.675345,0.728937
188,1.277530,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.959697,0.934231,1.228601,1.077496,-0.163231,0.658514,0.715410
189,1.282419,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,2.094602,0.957607,1.221214,1.067651,-0.155514,0.641683,0.701882
190,1.287308,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,2.203705,0.987746,1.213827,1.057806,-0.147797,0.624852,0.688355


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1212475533, 734209672, 3606247795, 3656790464, 2070314632, 100741134, 1037461789, 1666822053, 383049993, 4103241921, 1381659378, 3998629287, 4149728649, 1388935650, 4020125704, 3140267001, 3266608380, 149093451, 2635028298, 3461667241, 4216353172, 3887178911, 345758859, 3169975825, 1459637948, 574715990, 2030364251, 23693812, 1960285546, 137504185, 1928587921, 803067912, 2742094792, 4281292473, 228410119, 3444060420, 3877810207, 979599567, 3759701154, 1124040235, 3603905346, 4252031769, 2610965487, 3394472681, 2790178817, 692534677, 1511659990, 2639599435, 1456719936, 2616913005]


Step: 0 ___________________________________________
val_loss: 50.79209899902344
winner_seed: 1212475533


Step: 1 ___________________________________________
val_loss: 49.27178955078125
winner_seed: 734209672


Step: 2 ___________________________________________
val_loss: 54.16431427001953


Step: 3 ___________________________________________
val_loss: 42.37632751464844
winner_seed: 3656790464


Step: 4 ___

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 120ms/step - loss: 5119.7373 - val_loss: 393.5763
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 493.7821 - val_loss: 374.7060
Epoch 3/10000
4/4 [==============================] - 0s 9ms/step - loss: 399.3829 - val_loss: 342.6101
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 638.3652 - val_loss: 888.1096
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 483.1234 - val_loss: 272.8374
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 1201.5337 - val_loss: 3486.4409
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 2358.1150 - val_loss: 284.1181
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 247.7409 - val_loss: 169.3978
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 271.2889 - val_loss: 292.7230
Epoch 10/10000
4/4 [==============================] - 0s 10ms/step 

Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 83.6724 - val_loss: 75.4149
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 55.5221 - val_loss: 68.9372
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 45.8304 - val_loss: 63.1700
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 45.8588 - val_loss: 97.3208
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 50.0156 - val_loss: 90.1844
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 65.8843 - val_loss: 102.6655
Epoch 86/10000
4/4 [==============================] - 0s 12ms/step - loss: 100.2928 - val_loss: 80.0004
Epoch 87/10000
4/4 [==============================] - 0s 12ms/step - loss: 47.2894 - val_loss: 66.3414
Epoch 88/10000
4/4 [==============================] - 0s 9ms/step - loss: 42.6512 - val_loss: 66.3541
Epoch 89/10000
4/4 [==============================] - 0s 10ms/step - loss:

4/4 [==============================] - 0s 10ms/step - loss: 38.2490 - val_loss: 81.9377
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 46.2509 - val_loss: 70.6353
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 40.6685 - val_loss: 81.2984
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 40.2692 - val_loss: 99.9143
Epoch 163/10000
4/4 [==============================] - 0s 9ms/step - loss: 46.1095 - val_loss: 78.5189
Epoch 164/10000
4/4 [==============================] - 0s 9ms/step - loss: 42.6034 - val_loss: 63.5557
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 39.7225 - val_loss: 74.8082
Epoch 166/10000
4/4 [==============================] - 0s 8ms/step - loss: 39.5174 - val_loss: 62.3266
Epoch 167/10000
4/4 [==============================] - 0s 8ms/step - loss: 36.2859 - val_loss: 61.2532
Epoch 168/10000
4/4 [==============================] - 0s 8ms/step - loss: 46.3234 - v

4/4 [==============================] - 0s 9ms/step - loss: 36.1033 - val_loss: 67.0362
Epoch 239/10000
4/4 [==============================] - 0s 9ms/step - loss: 34.3530 - val_loss: 73.9974
Epoch 240/10000
4/4 [==============================] - 0s 9ms/step - loss: 42.1662 - val_loss: 72.6965
Epoch 241/10000
4/4 [==============================] - 0s 9ms/step - loss: 33.0305 - val_loss: 63.6049
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 32.4783 - val_loss: 69.3340
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 36.2063 - val_loss: 65.1777
Epoch 244/10000
4/4 [==============================] - 0s 8ms/step - loss: 31.0222 - val_loss: 88.3305
Epoch 245/10000
4/4 [==============================] - 0s 8ms/step - loss: 38.9323 - val_loss: 69.4168
Epoch 246/10000
4/4 [==============================] - 0s 9ms/step - loss: 32.1652 - val_loss: 62.0900
Epoch 247/10000
4/4 [==============================] - 0s 9ms/step - loss: 32.2236 - val_

4/4 [==============================] - 0s 9ms/step - loss: 31.0938 - val_loss: 68.0621
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 30.5201 - val_loss: 77.2130
Epoch 319/10000
4/4 [==============================] - 0s 8ms/step - loss: 38.9339 - val_loss: 79.2435
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 37.6138 - val_loss: 66.4534
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 26.6385 - val_loss: 65.8038
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - loss: 32.9916 - val_loss: 88.1784
Epoch 323/10000
4/4 [==============================] - 0s 9ms/step - loss: 34.1520 - val_loss: 86.7258
Epoch 324/10000
4/4 [==============================] - 0s 8ms/step - loss: 44.4894 - val_loss: 76.3702
Epoch 325/10000
4/4 [==============================] - 0s 9ms/step - loss: 34.9608 - val_loss: 78.8041
Epoch 326/10000
4/4 [==============================] - 0s 10ms/step - loss: 38.0492 - val

...

4/4 [==============================] - 0s 9ms/step - loss: 15.3421 - val_loss: 63.3145
Epoch 1888/10000
4/4 [==============================] - 0s 11ms/step - loss: 18.0829 - val_loss: 53.7023
Epoch 1889/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.5368 - val_loss: 59.5914
Epoch 1890/10000
4/4 [==============================] - 0s 9ms/step - loss: 19.0401 - val_loss: 59.5468
Epoch 1891/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.2185 - val_loss: 60.1482
Epoch 1892/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.6638 - val_loss: 57.3154
Epoch 1893/10000
4/4 [==============================] - 0s 8ms/step - loss: 16.4414 - val_loss: 56.8256
Epoch 1894/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.4453 - val_loss: 55.5726
Epoch 1895/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.5477 - val_loss: 59.1353
Epoch 1896/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.3

4/4 [==============================] - 0s 11ms/step - loss: 18.3176 - val_loss: 59.9219
Epoch 1966/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.6620 - val_loss: 58.6555
Epoch 1967/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.9973 - val_loss: 58.1321
Epoch 1968/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.4003 - val_loss: 59.8645
Epoch 1969/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.5204 - val_loss: 58.1063
Epoch 1970/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.4542 - val_loss: 57.2510
Epoch 1971/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.5359 - val_loss: 58.9998
Epoch 1972/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.0705 - val_loss: 59.2488
Epoch 1973/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.9493 - val_loss: 57.7525
Epoch 1974/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.6

4/4 [==============================] - 0s 9ms/step - loss: 16.6462 - val_loss: 59.5265
Epoch 2044/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.0542 - val_loss: 60.1694
Epoch 2045/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.8857 - val_loss: 58.4886
Epoch 2046/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.9207 - val_loss: 58.1797
Epoch 2047/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.6440 - val_loss: 58.1483
Epoch 2048/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.3372 - val_loss: 59.7691
Epoch 2049/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.5429 - val_loss: 61.9714
Epoch 2050/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.4111 - val_loss: 58.8965
Epoch 2051/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.2909 - val_loss: 58.5552
Epoch 2052/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.8

4/4 [==============================] - 0s 10ms/step - loss: 15.1069 - val_loss: 61.1346
Epoch 2122/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.2606 - val_loss: 59.1045
Epoch 2123/10000
4/4 [==============================] - 0s 10ms/step - loss: 16.9391 - val_loss: 57.9356
Epoch 2124/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.9204 - val_loss: 59.1816
Epoch 2125/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.6796 - val_loss: 59.6201
Epoch 2126/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.4194 - val_loss: 58.6304
Epoch 2127/10000
4/4 [==============================] - 0s 10ms/step - loss: 16.9607 - val_loss: 60.3353
Epoch 2128/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.6437 - val_loss: 59.1302
Epoch 2129/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.3352 - val_loss: 60.6736
Epoch 2130/10000
4/4 [==============================] - 0s 9ms/step - los

4/4 [==============================] - 0s 10ms/step - loss: 14.7818 - val_loss: 59.4856
Epoch 2200/10000
4/4 [==============================] - 0s 10ms/step - loss: 16.3984 - val_loss: 64.3263
Epoch 2201/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.1976 - val_loss: 63.8585
Epoch 2202/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.6069 - val_loss: 63.2363
Epoch 2203/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.1358 - val_loss: 60.3761
Epoch 2204/10000
4/4 [==============================] - 0s 8ms/step - loss: 13.2205 - val_loss: 71.8637
Epoch 2205/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.6193 - val_loss: 74.2297
Epoch 2205: early stopping


In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,65.463165,65.50309,65.412018,65.29525,65.227173,65.17746,65.105072,65.076614,64.959824,64.777367,64.585518,64.466919,64.109688,64.00489,63.907608,63.289928,61.656464,58.442963,56.290264,55.995232,55.668926,55.197968,55.351471,55.249229,56.240234,56.937927,58.482475,52.955559,54.713894,57.764328
Target,61.721,69.24,70.048,67.172,64.22,63.021,62.639,53.839,56.652,53.408,50.543,64.699,60.219,64.217,57.591,64.307,61.971,57.48,68.053,55.231,53.723,57.457,62.755,48.148,62.19,62.219,61.329,81.988,79.341,76.646
Error,3.742165,3.736908,4.635979,1.876747,1.007172,2.15646,2.466072,11.237614,8.307823,11.369366,14.042519,0.232079,3.890686,0.212112,6.316608,1.017071,0.314537,0.962963,11.762737,0.764233,1.945927,2.259033,7.40353,7.101231,5.949764,5.281075,2.846523,29.03244,24.62711,18.881676


In [38]:
display(mae)
display(mape)

6.512672

0.10094667

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined